# AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [ ]:
# Import the GrpcWorkerAgentRuntimeHost class, which provides the gRPC-based host for distributed agent runtimes
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

# Create an instance of the gRPC host, specifying the address (host:port) it should listen on for worker connections
host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")

# Start the host service so it begins listening for incoming gRPC connections from worker runtimes
host.start() 

### Let's reintroduce a tool

In [ ]:
# Create an instance of the GoogleSerperAPIWrapper, which provides a search interface using the Serper API
serper = GoogleSerperAPIWrapper()

# Wrap the Serper API as a LangChain Tool, giving it a name and description for agent use
langchain_serper = Tool(
    name="internet_search", 
    func=serper.run, 
    description="Useful for when you need to search the internet"
)

# Adapt the LangChain Tool for use with AutoGen agents by wrapping it with LangChainToolAdapter
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [ ]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime  # Import the gRPC-based worker runtime for agent execution

if ALL_IN_ONE_WORKER:  # Check if all agents should be run in a single worker process

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")  # Create a worker runtime connected to the host at the specified address
    await worker.start()  # Start the worker runtime asynchronously

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))  # Register Player1Agent with the worker under the name "player1"
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))  # Register Player2Agent with the worker under the name "player2"
    await Judge.register(worker, "judge", lambda: Judge("judge"))  # Register Judge agent with the worker under the name "judge"

    agent_id = AgentId("judge", "default")  # Create an AgentId for the judge agent in the default namespace

else:  # If agents should be distributed across multiple workers

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")  # Create the first worker runtime for player1
    await worker1.start()  # Start the first worker asynchronously
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))  # Register Player1Agent with worker1

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")  # Create the second worker runtime for player2
    await worker2.start()  # Start the second worker asynchronously
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))  # Register Player2Agent with worker2

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")  # Create a separate worker runtime for the judge
    await worker.start()  # Start the judge's worker asynchronously
    await Judge.register(worker, "judge", lambda: Judge("judge"))  # Register Judge agent with its worker
    agent_id = AgentId("judge", "default")  # Create an AgentId for the judge agent in the default namespace



In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are several pros of using AutoGen in your new AI Agent project:

1. **Multi-Agent Collaboration**: AutoGen excels at facilitating interaction among multiple agents, allowing them to collaborate effectively on complex tasks, which can lead to better outcomes compared to single-agent systems.

2. **Efficiency**: It streamlines complex tasks and automates workflows, making it easier to manage and execute intricate processes through the cooperation of autonomous agents.

3. **Scalability**: AutoGen's architecture allows agents to handle multiple tasks simultaneously, making it highly scalable and suitable for projects with growing demands.

4. **Optimization with LLMs**: AutoGen can integrate with various large language models (LLMs), enhancing the problem-solving capabilities of the agents by leveraging specialized tools tailored for different tasks.

5. **Customization and Adaptability**: The agents created using AutoGen can be tailored to specific needs and can adapt based on ongoing tasks or human feedback, ensuring flexibility in diverse applications.

6. **Dynamic Collaboration**: AutoGen fosters an environment where agents can develop collaborative strategies, which enhances their performance and ability to tackle more complicated issues than individual agents might manage alone.

These features make AutoGen a compelling choice for projects requiring sophisticated AI agent capabilities. 

TERMINATE

## Cons of AutoGen:
Here are some cons of using AutoGen for your AI agent project:

1. **Complexity and Learning Curve**: AutoGen requires a solid understanding of multi-agent systems, which can be a significant barrier for newcomers or those without programming expertise.

2. **Challenging Documentation**: The documentation for AutoGen is often described as hard to read with insufficient examples, making it difficult for users to learn and utilize the framework effectively.

3. **Customization Complexity**: While AutoGen offers extensive customization options, this flexibility can also lead to increased complexity, possibly overwhelming developers who are new to such extensive configuration.

4. **Interaction Management**: AutoGen lacks an inherent concept of process management, which can complicate handling interactions among agents within the system, leading to potential confusion or errors in coordination.

These factors could be critical in deciding whether AutoGen is the right fit for your project. 

TERMINATE



## Decision:

Based on the research from the team, I recommend using AutoGen for the project. The advantages of multi-agent collaboration, efficiency, scalability, optimization with LLMs, customization, and dynamic interaction outweigh the challenges of complexity and documentation issues. The ability to tackle complex tasks through collaboration among agents is particularly compelling for sophisticated AI capabilities. While there may be a learning curve, the potential for enhanced outcomes justifies the initial investment in usability training and overcoming documentation challenges.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()